In [145]:
import GUI
import nltk
from nltk.stem.snowball import SnowballStemmer  # type: ignore
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np # type: ignore
import pandas as pd # type: ignore
import re
from icecream import ic

In [2]:
# function to create the excel files
def create_excel_dataframe(input , name, columns, index):
    df = pd.DataFrame(input, columns = columns, index = index) # Create a pandas dataframe
    df.to_excel(name) # Create an excel file
    return df

In [3]:
# Download the Stop words
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/glados/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def convert_to_list(doc) -> list:
    doc = re.sub(r"[^\w\s]", " ", doc)
    doc = doc.split()
    return doc

In [5]:
stemmer = SnowballStemmer(language='english')
dictionary = []

In [6]:
nltk.download('wordnet')

def is_meaningful(word) -> bool:
    
    if wn.synsets(word):
        return True
    else: 
        return False    

[nltk_data] Downloading package wordnet to /home/glados/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def create_dictionary(doc, dictionary) -> list:
    for word_index, word in enumerate(doc):
        
        word = word.lower()
        
        if word.isalnum() and word not in stop_words and is_meaningful(word) and len(word) > 1:
            
            stem_word = stemmer.stem(word)
            doc[word_index] = stem_word
            
            if stem_word not in dictionary:
                dictionary.append(stem_word)
    
    return doc

In [8]:
def join_document(doc) -> str:
    doc = ' '.join(doc)
    return doc

In [9]:
def process_document(document) -> str:
    
    document = convert_to_list(document)
    
    document = create_dictionary(document, dictionary)
    
    document = join_document(document)
    
    return document

In [10]:
raw_document = GUI.select_file() # Select File GUI implementation

raw_dataset = raw_document # the raw dataset that read from the one file

raw_dataset = raw_dataset.split(".I") # split the docs by index

raw_dataset.remove(raw_dataset[0]) # remove the empty first index

doc_index = 0
    

# write each index in 'raw_dataset'
for doc in raw_dataset:
    doc_index += 1
    doc_path = "./docs" + "/doc" + str(doc_index) + ".txt"
    doc = process_document(doc)
    
    
    with open(doc_path, "w") as document: # write each index into the separated doc
        document.write(doc)

doc_index = 0

for doc in raw_dataset:
    doc_index += 1
    doc_path = "./Original docs" + "/docs" + str(doc_index) + ".txt"
    
    
    with open(doc_path, "w") as document: # write each index into the separated doc
        document.write(doc)

GUI.custom_window("700x35", "Extract Documents", f"There have been {doc_index} documents are extracted.")

In [11]:
df = pd.DataFrame(dictionary, columns=["Words"])
df

,Words
0,experiment
1,investig
2,aerodynam
3,wing
4,slipstream
...,...
3859,hopkin
3860,aeolotrop
3861,intuit
3862,recover


In [12]:
df.to_excel("dictionary.xlsx")

In [13]:
term_doc_matrix = np.zeros((len(dictionary), doc_index))
tf_array = np.zeros((len(dictionary), doc_index))
idf_array = np.zeros((len(dictionary), 1))
tf_idf_array = np.zeros((len(dictionary), doc_index))

In [14]:
def calculate_term_frequency(doc, dictionary, doc_no):
    for word in dictionary:
        if word in doc:
            frequency = doc.count(word)
            row = dictionary.index(word)
            column = doc_no - 1
            tf_array[row, column] += frequency


def calculate_inverse_document_frequency(doc, dictionary):
    for word in dictionary:
        if word in doc:
            row = dictionary.index(word)
            idf_array[row, 0] += 1

In [15]:
doc_names_list = []
for num in range(1, doc_index + 1):
    doc_path = "./docs" + "/doc" + str(num) + ".txt"
    doc_names_list.append("doc" + str(num) + ".txt")
    with open(doc_path, "r") as file:
        doc = file.read()
        calculate_term_frequency(doc, dictionary, num)
        calculate_inverse_document_frequency(doc, dictionary)

In [16]:
# calculating tf array
row, column = np.shape(tf_array)
for r in range(row):
    for c in range(column):
        if tf_array[r, c] > 0:
            extracted_element = tf_array[r, c]
            tf_array[r, c] = 1 + np.log10(extracted_element)
        else:
            tf_array[r, c] = 0
            
GUI.custom_window("700x35", "Complete!", "The calculation of Term Frequency is have been successful.")

In [17]:
count_of_documents = column
row, column = np.shape(idf_array)


# calculating idf array
for r in range(row):
    if idf_array[r, 0] > 0:
        extracted_element = idf_array[r, 0]
        idf_array[r, 0] = np.log10(count_of_documents / extracted_element)
    else:
        idf_array[r, 0] = 0

GUI.custom_window("700x35", "Complete!", "The calculation of Inverse Document Frequency is have been successful.")

In [18]:
# multiply peer to peer each row of tf-array in idf-array
row, column = np.shape(tf_idf_array)
r, c = 0, 0
while r < row:
        tf_idf_array[r, c]= tf_array[r, c] * idf_array[r, 0]
        c += 1
        if c == column:
            r += 1
            c = 0

In [19]:
tf_dataframe = create_excel_dataframe(tf_array, "tf_excel.xlsx", doc_names_list, dictionary)

idf_dataframe = create_excel_dataframe(idf_array, "idf_excel.xlsx", ["IDF"], dictionary)

tf_idf_dataframe = create_excel_dataframe(tf_idf_array, "tf_idf_excel.xlsx", doc_names_list, dictionary)

GUI.custom_window("700x35", "Excel File", "Creating the Excel file is DONE!")

In [20]:
tf_dataframe

,doc1.txt,doc2.txt,doc3.txt,doc4.txt,doc5.txt,doc6.txt,doc7.txt,doc8.txt,doc9.txt,doc10.txt,...,doc1391.txt,doc1392.txt,doc1393.txt,doc1394.txt,doc1395.txt,doc1396.txt,doc1397.txt,doc1398.txt,doc1399.txt,doc1400.txt
experiment,1.477121,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,1.000000,0.0,0.000000,0.0,1.0,1.0,0.00000,0.0,0.0
investig,1.301030,1.00000,0.0,0.00000,0.0,0.00000,1.0,1.00000,1.477121,0.0,...,0.0,0.000000,1.0,0.000000,1.0,0.0,0.0,0.00000,0.0,1.0
aerodynam,1.301030,0.00000,0.0,0.00000,1.0,0.00000,0.0,0.00000,0.000000,0.0,...,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0
wing,1.602060,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0
slipstream,1.778151,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hopkin,0.000000,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0
aeolotrop,0.000000,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0
intuit,0.000000,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,1.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0
recover,0.000000,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.0,...,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.00000,0.0,0.0


In [21]:
idf_dataframe

,IDF
experiment,0.615928
investig,0.586221
aerodynam,0.876615
wing,0.788193
slipstream,1.970037
...,...
hopkin,3.146128
aeolotrop,3.146128
intuit,3.146128
recover,1.714764


In [22]:
tf_idf_dataframe

,doc1.txt,doc2.txt,doc3.txt,doc4.txt,doc5.txt,doc6.txt,doc7.txt,doc8.txt,doc9.txt,doc10.txt,...,doc1391.txt,doc1392.txt,doc1393.txt,doc1394.txt,doc1395.txt,doc1396.txt,doc1397.txt,doc1398.txt,doc1399.txt,doc1400.txt
experiment,0.909801,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.615928,0.000000,0.000000,0.000000,0.615928,0.615928,0.000000,0.0,0.000000
investig,0.762692,0.586221,0.0,0.000000,0.000000,0.000000,0.586221,0.586221,0.865920,0.0,...,0.000000,0.000000,0.586221,0.000000,0.586221,0.000000,0.000000,0.000000,0.0,0.586221
aerodynam,1.140503,0.000000,0.0,0.000000,0.876615,0.000000,0.000000,0.000000,0.000000,0.0,...,0.876615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
wing,1.262733,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
slipstream,3.503023,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hopkin,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,3.146128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
aeolotrop,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,3.146128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
intuit,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,3.146128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
recover,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,1.714764,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [95]:
query = input("Please enter what you want to search")

In [125]:
query_lower = query.lower()

In [126]:
query_words_list = query_lower.split()

In [127]:
# remove the stop words
for word in query_words_list:
    if word in stop_words or not is_meaningful(word):
        query_words_list.remove(word)  
query_words_list

['oh',
 'hell',
 'night',
 'city',
 'wake',
 'fuck',
 'samurai',
 'have',
 'city',
 'burn']

In [128]:
temp = [stemmer.stem(word) for word in query_words_list]
query_words = set(temp) 
query_words_list = temp
query_words_list

['oh',
 'hell',
 'night',
 'citi',
 'wake',
 'fuck',
 'samurai',
 'have',
 'citi',
 'burn']

In [129]:
query_words_list = " ".join(query_words_list)
query_words_list

'oh hell night citi wake fuck samurai have citi burn'

In [130]:
query_words_list = re.sub(r"[^\w\s]"," ", query_words_list)
query_words_list

'oh hell night citi wake fuck samurai have citi burn'

In [131]:
string = [query_words_list]

In [133]:
query_words

{'burn', 'citi', 'fuck', 'have', 'hell', 'night', 'oh', 'samurai', 'wake'}

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer



tfidf = TfidfVectorizer()
 

result = tfidf.fit_transform(string)

query_tfidf = np.squeeze(result.toarray())
query_tfidf

array([0.28867513, 0.57735027, 0.28867513, 0.28867513, 0.28867513,
       0.28867513, 0.28867513, 0.28867513, 0.28867513])

In [188]:
query_words = list(tfidf.vocabulary_.keys())
query_words

['oh', 'hell', 'night', 'citi', 'wake', 'fuck', 'samurai', 'have', 'burn']

#### Make the values of the words in query which is not present in dictionary documents to Zero(0).
for corresponding the query words and the dictionary, we obtain the indexes of the words in the dictionary, to for later change the rows of the tf_idf array.

In [184]:
query_words_in_dictionary = {}

for index, word in enumerate(query_words):
    
    if word not in dictionary:
        query_tfidf[index] = 0
    else:
        query_words_in_dictionary.update({word : dictionary.index(word)})
   
      
query_words_in_dictionary

{'night': 2908, 'citi': 3355, 'wake': 546, 'burn': 1521}

In [173]:
query_tfidf

array([0.        , 0.        , 0.28867513, 0.28867513, 0.28867513,
       0.        , 0.        , 0.        , 0.28867513])

### Add pad 0 to query_tfidf to make it equal to the tf-idf of the words.

In [ ]:
row, column = np.shape(tf_idf_array)

In [174]:
query_tfidf = np.pad(query_tfidf, (0, row - len(query_tfidf) + 1), 'constant', constant_values = (0))

3864

### Move the values to the Corresponding index.

In [191]:
for count, value in enumerate(query_tfidf):
    if count < len(query_words):
        if value > 0:
            if query_words[count] != dictionary.index(query_words[count]):
                dest_index = dictionary.index(query_words[count])
                query_tfidf[dest_index] = value
                query_tfidf[count] = 0

array([0., 0., 0., ..., 0., 0., 0.])

In [196]:
query_tfidf

array([0., 0., 0., ..., 0., 0., 0.])

In [197]:
def change_shape(document):
    document = np.array(document)
    row = np.shape(document)
    row = row[0]
    document.shape = (1, row)
    return document

In [69]:

cosine_values = []


for number in range(1, column):
    
    doc_column = tf_idf_dataframe["doc"+ str(number) +".txt"].tolist()
    
    doc_column = change_shape(doc_column)
    
    nominator = np.dot(query_tfidf, doc_column)
    
    doc_column_norm = np.linalg.norm(doc_column)
    query_tfidf_norm = np.linalg.norm(query_tfidf)
    
    denominator = doc_column_norm * query_tfidf_norm
    
    cosine_theta = nominator / denominator
    cosine_values.append(cosine_theta)

cosine_values = np.squeeze(cosine_values)
# df = pd.DataFrame(cosine_values, index = doc_names_list, columns = ['cosine'])
# df
# sorted(cosine_values)
np.shape(cosine_values)

[[0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]
 [0.67082039]
 [0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]
 [0.2236068 ]]
[0.9098008386686979, 0.7626916393458575, 1.1405025286416028, 1.2627327730178903, 3.5030233576531624, 0.6146491186359828, 0.958607314841775, 0.765916793966632, 1.5898255349109507, 0.7800785541936918, 0.851661809516645, 0.6675615400843947, 0.5874194651450724, 1.509153413255798, 0.8259817495671841, 1.289050109139196, 1.0347586422825692, 0.7732160327081314, 1.0892231843417655, 0.7361949123469436, 1.1017537495100926, 0.6396230032733659, 0.6632544520694843, 0.30602194122148024, 2.066946789630613, 0.8666435223746027, 1.692679049617419, 1.1127042801912883, 0.7750601734065017, 1.5333441789585025, 0.6369255133471352, 0.5910335870999189, 1.8087610887547128, 0.8364978682523392, 0.9312841876305402, 1.5333441789585025, 1.3010299956639813, 1.5898255349109507, 0.49291552190289434, 1.5333441789585025, 2.4923559897712617, 1.1332908109730

(1399, 12, 3864)